## Method3 (Basically Method2 but without bbox)

In [1]:
from utils import *

#### Load a file

In [2]:
mesh = pymesh.load_mesh("data/two_spheres2.ply")

In [3]:
self_intersection_stats(mesh)

+-----------------------------------+---------+
| Metric                            |   Value |
+===================================+=========+
| Number of vertices                |     952 |
+-----------------------------------+---------+
| Number of faces                   |    1896 |
+-----------------------------------+---------+
| Number of intersecting face pairs |     119 |
+-----------------------------------+---------+


In [4]:
visualize_intersection(mesh)

/home/kmaeda/anaconda3/envs/pymesh_env/lib/python3.10/site-packages/pyvista/core/pointset.py:1365: PyVistaDeprecationWarning: The current behavior of `pv.PolyData.n_faces` has been deprecated.
                Use `pv.PolyData.n_cells` or `pv.PolyData.n_faces_strict` instead.
                See the documentation in '`pv.PolyData.n_faces` for more information.
  warnings.warn(


Widget(value='<iframe src="http://localhost:46737/index.html?ui=P_0x7f63e7c22b30_0&reconnect=auto" class="pyvi…

None

#### Save intersection info first

In [5]:
intersections = pymesh.detect_self_intersection(mesh)
intersecting_vertices, intersecting_faces = track_self_intersecting_faces(mesh, intersections)

#### Get outer hull

In [6]:
outer_hull = pymesh.compute_outer_hull(mesh)

In [7]:
self_intersection_stats(outer_hull)

+-----------------------------------+---------+
| Metric                            |   Value |
+===================================+=========+
| Number of vertices                |     917 |
+-----------------------------------+---------+
| Number of faces                   |    1830 |
+-----------------------------------+---------+
| Number of intersecting face pairs |       0 |
+-----------------------------------+---------+


#### Map intersecting region to modified mesh & Extract submesh

In [8]:
mapped_vertices = map_to_modified_mesh(mesh, outer_hull, intersecting_vertices)

In [9]:
submesh, face_mask = extract_self_intersecting_region_from_modified(outer_hull, mapped_vertices)

In [10]:
visualize(submesh, filename="Submesh")

Widget(value='<iframe src="http://localhost:46737/index.html?ui=P_0x7f63e0bafc10_1&reconnect=auto" class="pyvi…

None

In [11]:
remaining_mesh = extract_remaining_mesh(outer_hull, face_mask)

In [12]:
visualize(remaining_mesh, filename="remaining_mesh")

Widget(value='<iframe src="http://localhost:46737/index.html?ui=P_0x7f63e0bafa60_2&reconnect=auto" class="pyvi…

None

#### Repair (or Remesh) the submesh
- `pymesh.collapse_short_edges(mesh, abs_threshold, rel_threshold, preserve_feature)`<br>
Collapses all edges with length less than a user specified threshold.

- `pymesh.split_long_edges(mesh, max_edge_length)`<br>
Splits long edges into 2 or more shorter edges.

- `pymesh.remove_obtuse_triangles(mesh, max_angle, max_iterations)`<br>
Splits each obtuse triangle into 2 or more right or sharp triangles.

- `pymesh.remove_degenerated_triangles(mesh, num_iterations)` <br>
Degenerate triangles are triangles with collinear vertices (i.e., some vertices are lying in the same straight line). They have zero areas and their normals are undefined.

In [13]:
from numpy.linalg import norm

def method2(mesh, detail=2e-2):

    bbox_min, bbox_max = mesh.bbox
    diag_len = norm(bbox_max - bbox_min)
    target_len = diag_len * detail
    print("Target resolution: {} mm".format(target_len))

    count = 0
    mesh, __ = pymesh.remove_degenerated_triangles(mesh, 100)
    mesh, __ = pymesh.split_long_edges(mesh, target_len)
    num_vertices = mesh.num_vertices
    while True:
        mesh, __ = pymesh.collapse_short_edges(mesh, 1e-6, preserve_feature=True) # Remove extremely small edges
        mesh, __ = pymesh.collapse_short_edges(mesh, target_len,
                                               preserve_feature=True)
        mesh, __ = pymesh.remove_obtuse_triangles(mesh, 150.0, 100)
        if mesh.num_vertices == num_vertices:
            break

        num_vertices = mesh.num_vertices
        print("#v: {}".format(num_vertices))
        count += 1
        if count > 10: break

    mesh = pymesh.resolve_self_intersection(mesh)
    mesh, __ = pymesh.remove_duplicated_faces(mesh)
    mesh = pymesh.compute_outer_hull(mesh)
    mesh, __ = pymesh.remove_duplicated_faces(mesh)
    mesh, __ = pymesh.remove_obtuse_triangles(mesh, 179.0, 5)
    mesh, __ = pymesh.remove_isolated_vertices(mesh)

    return mesh

In [14]:
repaired_submesh = method2(submesh)

Target resolution: 0.058262021377178466 mm
#v: 428
#v: 427


In [15]:
visualize(repaired_submesh, filename="Repaired_submesh")

Widget(value='<iframe src="http://localhost:46737/index.html?ui=P_0x7f63e0baff70_3&reconnect=auto" class="pyvi…

None

#### Align submesh
Since we have modified the submesh, we need to fix its boundary. 

In [16]:
aligned_submesh = align_submesh_boundary(remaining_mesh, repaired_submesh)

In [17]:
visualize(aligned_submesh, filename="Method2")

Widget(value='<iframe src="http://localhost:46737/index.html?ui=P_0x7f63c25d29b0_4&reconnect=auto" class="pyvi…

None

#### Merge two meshes

In [18]:
repaired_full = replace_submesh_in_original(remaining_mesh, aligned_submesh)

In [19]:
visualize(repaired_full)

Widget(value='<iframe src="http://localhost:46737/index.html?ui=P_0x7f636af0e3b0_5&reconnect=auto" class="pyvi…

None

#### Final refinement

In [20]:
final = refinement(repaired_full)

In [21]:
self_intersection_stats(final)

+-----------------------------------+---------+
| Metric                            |   Value |
+===================================+=========+
| Number of vertices                |    1021 |
+-----------------------------------+---------+
| Number of faces                   |    2040 |
+-----------------------------------+---------+
| Number of intersecting face pairs |       0 |
+-----------------------------------+---------+


In [22]:
visualize(final, filename="Final Outerhull")

Widget(value='<iframe src="http://localhost:46737/index.html?ui=P_0x7f636af0da20_6&reconnect=auto" class="pyvi…

None

In [23]:
evaluation(outer_hull, final)

+-----------------------------------+------------+-----------+
| Metric                            |     Before |     After |
+===================================+============+===========+
| Number of vertices                |  917       | 1021      |
+-----------------------------------+------------+-----------+
| Number of faces                   | 1830       | 2040      |
+-----------------------------------+------------+-----------+
| Number of intersecting face pairs |    0       |    0      |
+-----------------------------------+------------+-----------+
| Volume                            |    6.61368 |    6.6111 |
+-----------------------------------+------------+-----------+
| Area                              |   19.0834  |   19.0837 |
+-----------------------------------+------------+-----------+


In [24]:
evaluate_displacement(outer_hull, final)

Max Vertex Displacement: 0.11448359749842713
Mean Vertex Displacement: 0.010789851710345038


In [25]:
pymesh.save_mesh("temp.ply", final, ascii=True)